## Part 0: Environment

In [ ]:
!pip install datasets torch torchvision torchaudio tqdm pandas huggingface_hub
!pip install transformers==4.30
!pip install sentencepiece
!pip install protobuf cpm_kernels gradio mdtex2html sentencepiece accelerate
!pip install loguru
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate -U

## Part 1: Preparing the Data
Data preparation is a crucial step when it comes to training Financial Large Language Models. Here, we’ll guide you on how to get your dataset ready for FinGPT using Python.

In this section, you’ve initialized your working directory and loaded a financial sentiment dataset. Let’s break down the steps:



### 1.1 Initialize Directories:
This block checks if certain paths exist; if they do, it deletes them to avoid data conflicts, and then creates a new directory for the upcoming data.



In [ ]:
import os
import shutil

jsonl_path = "../data/dataset_new.jsonl"
save_path = '../data/dataset_new'


if os.path.exists(jsonl_path):
    os.remove(jsonl_path)

if os.path.exists(save_path):
    shutil.rmtree(save_path)

directory = "../data"
if not os.path.exists(directory):
    os.makedirs(directory)

### 1.2 Load and Prepare Dataset:

* Import necessary libraries from the datasets package: https://huggingface.co/docs/datasets/index
* Load the Twitter Financial News Sentiment (TFNS) dataset and convert it to a Pandas dataframe. https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment
* Map numerical labels to their corresponding sentiments (negative, positive, neutral).
* Add instruction for each data entry, which is crucial for Instruction Tuning.
* Convert the Pandas dataframe back to a Hugging Face Dataset object.




In [ ]:
from datasets import load_dataset
import datasets

dic = {
    0:"negative",
    1:'positive',
    2:'neutral',
}

tfns = load_dataset('zeroshot/twitter-financial-news-sentiment')
tfns = tfns['train']
tfns = tfns.to_pandas()
tfns['label'] = tfns['label'].apply(lambda x:dic[x])
tfns['instruction'] = 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.'
tfns.columns = ['input', 'output', 'instruction']
tfns = datasets.Dataset.from_pandas(tfns)
tfns

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9543
})

### 1.3 Concatenate and Shuffle Dataset

In [ ]:
tmp_dataset = datasets.concatenate_datasets([tfns]*2)
train_dataset = tmp_dataset
print(tmp_dataset.num_rows)

all_dataset = train_dataset.shuffle(seed = 42)
all_dataset.shape

19086


(19086, 3)

Now that your training data is loaded and prepared.

## Part 2: Dataset Formatting and Tokenization
Once your data is prepared, the next steps involve formatting the dataset for model ingestion and tokenizing the input data. Below, we provide a step-by-step breakdown of the code snippets shared.



### 2.1 Dataset Formatting:
You need to structure your data in a specific format that aligns with the training process.

In [ ]:
import json
from tqdm.notebook import tqdm

In [ ]:
def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [ ]:
data_list = []
for item in all_dataset.to_pandas().itertuples():
    tmp = {}
    tmp["instruction"] = item.instruction
    tmp["input"] = item.input
    tmp["output"] = item.output
    data_list.append(tmp)

In [ ]:
# save to a jsonl file
with open("../data/dataset_new.jsonl", 'w') as f:
    for example in tqdm(data_list, desc="formatting.."):
        f.write(json.dumps(format_example(example)) + '\n')

formatting..:   0%|          | 0/19086 [00:00<?, ?it/s]

In [ ]:
# print first few data in data_list
print("raw data:")
print(data_list[:3])

# See examples of data
print("\n data after formatting:")
for example in data_list[:3]:
    formatted_example = format_example(example)
    print(formatted_example)


原始数据:
[{'instruction': 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.', 'input': '$DRIP $LABU $GASX - SOXL, LABU, JO and GUSH among weekly ETF movers https://t.co/FntrWNY9sn', 'output': 'neutral'}, {'instruction': 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.', 'input': 'From Farms to Silicon Valley, U.S. Businesses Stand to Gain From USMCA -- Update #economy #MarketScreener… https://t.co/fpxwDcClZh', 'output': 'neutral'}, {'instruction': 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.', 'input': 'Broadridge acquires Clearstructure Financial Technology', 'output': 'neutral'}]

处理后的数据:
{'context': 'Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.\nInput: $DRIP $LABU $GASX - SOXL, LABU, JO and GUSH among weekly ETF movers https://t.co/FntrWNY9sn\nAnswer: ', 'target': 'neutral'}
{'conte

### 2.2 Tokenization
Tokenization is the process of converting input text into tokens that can be fed into the model.
分割处理好的数据


In [ ]:
import datasets
from transformers import AutoTokenizer, AutoConfig

model_name = "baichuan-inc/Baichuan2-7B-Base"
jsonl_path = "../data/dataset_new.jsonl"  # updated path
save_path = '../data/dataset_new'  # updated path
max_seq_length = 512 #模型能够处理的最大令牌数，如果以后发现报错可能是这里
skip_overlength = True #跳过超过最大令牌数的数据条目

In [ ]:
# The preprocess function tokenizes the prompt and target, combines them into input IDs,
# and then trims or pads the sequence to the maximum sequence length.
def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True) #truncation会截断超出部分的数据条目
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id] #变成令牌
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

# The read_jsonl function reads each line from the JSONL file, preprocesses it using the preprocess function,
# and then yields each preprocessed example.
def read_jsonl(path, max_seq_length, skip_overlength=False):#这里的skip_overlength函数设置为false，我们将超出的长度进行截取，而不跳过
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)#安全参数，代表信任这个第三方远程代码
    config = AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')#优化模式
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:#两个都是true则跳过
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]#跳过失败的进行截取
            yield feature

### 2.3 Save the dataset

In [ ]:
# The script then creates a Hugging Face Dataset object from the generator and saves it to disk.
save_path = '../data/dataset_new'

dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(jsonl_path, max_seq_length, skip_overlength)
    )
dataset.save_to_disk(save_path)


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

tokenization_baichuan.py:   0%|          | 0.00/9.63k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan2-7B-Base:
- tokenization_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

configuration_baichuan.py:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan2-7B-Base:
- configuration_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


  0%|          | 0/19086 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/19086 [00:00<?, ? examples/s]

## Part 3: Setup FinGPT training parameters with LoRA on ChatGlm2-6b

Training a model can be resource-intensive. Ensure you have a powerful GPU
 * Need to purchase Google Colab GPU plans, Colab Pro is sufficient or just buy 100 compute units for $10
 * NVIDIA A100 is recommended due to its high memory capacity.



In [ ]:
# Ensure CUDA is accessible in the system path
# Only for Windows Subsystem for Linux (WSL)
import os
os.environ["PATH"] = f"{os.environ['PATH']}:/usr/local/cuda/bin"
os.environ['LD_LIBRARY_PATH'] = "/usr/lib/wsl/lib:/usr/local/cuda/lib64"

### 3.1 Training Arguments Setup:
Initialize and set training arguments.



In [ ]:
from typing import List, Dict, Optional
import torch
import bitsandbytes
from loguru import logger
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training,
    prepare_model_for_int8_training,
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

In [ ]:
training_args = TrainingArguments(
        output_dir='./finetuned_model',    # saved model path
        logging_steps = 500,
        # max_steps=10000,
        num_train_epochs = 2,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        weight_decay=0.01,
        warmup_steps=1000,
        save_steps=500,
        fp16=True,
        #bf16=True,
        torch_compile = False,
        load_best_model_at_end = True,
        evaluation_strategy="steps",
        remove_unused_columns=False,

    )

### 3.2 Quantization Config Setup:
Set quantization configuration to reduce model size without losing significant precision.
这部分减小了了训练的模型大小


In [ ]:
 # Quantization
q_config = BitsAndBytesConfig(load_in_4bit=True,
                bnb_4bit_quant_type='nf4',
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16
                                )

### 3.3 Model Loading & Preparation:
Load the base model and tokenizer, and prepare the model for INT8 training.

* **Runtime -> Change runtime type -> A100 GPU**
* retart runtime and run again if not working


In [ ]:
# Load tokenizer & model
# need massive space
model_name = "baichuan-inc/Baichuan2-7B-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=q_config,
        trust_remote_code=True,
        device='cuda'
    ) #q_config reduce the size of model
model = prepare_model_for_int8_training(model, use_gradient_checkpointing=True)#减少运算量

modeling_baichuan.py:   0%|          | 0.00/33.1k [00:00<?, ?B/s]

generation_utils.py:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan2-7B-Base:
- generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


quantizer.py:   0%|          | 0.00/9.07k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan2-7B-Base:
- quantizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan2-7B-Base:
- modeling_baichuan.py
- generation_utils.py
- quantizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
pip install xformers.


pytorch_model.bin.index.json:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:

print(model)


BaichuanForCausalLM(
  (model): BaichuanModel(
    (embed_tokens): Embedding(125696, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): NormHead()
)


### 3.4 LoRA Config & Setup:
Implement Low-Rank Adaptation (LoRA) and print trainable parameters.



In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
target_modules = ['W_pack']

In [ ]:
# LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules,
    bias='none',
)
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 4194304 || all params: 4272164864 || trainable%: 0.0981774845662885


In [ ]:
resume_from_checkpoint = None
if resume_from_checkpoint is not None:
    checkpoint_name = os.path.join(resume_from_checkpoint, 'pytorch_model.bin')
    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, 'adapter_model.bin'
        )
        resume_from_checkpoint = False
    if os.path.exists(checkpoint_name):
        logger.info(f'Restarting from {checkpoint_name}')
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        logger.info(f'Checkpoint {checkpoint_name} not found')

In [ ]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 7,510,167,552 || trainable%: 0.055848341211549045


## Part 4: Loading Data and Training FinGPT
In this segment, we'll delve into the loading of your pre-processed data, and finally, launch the training of your FinGPT model. Here's a stepwise breakdown of the script provided:
* Need to purchase Google Colab GPU plans, Colab Pro is sufficient or just buy 100 compute units for $10


### 4.1 Loading Your Data:


In [ ]:
# load data
from datasets import load_from_disk
import datasets

dataset = datasets.load_from_disk("../data/dataset_new")
dataset = dataset.train_test_split(0.2, shuffle=True, seed = 42)

### 4.2 Training Configuration and Launch:
* Customize the Trainer class for specific loss computation, prediction step, and model-saving methods.

* Define a data collator function to process batches of data during training.

* Set up TensorBoard for logging, instantiate your modified trainer, and begin training.



In [ ]:
class ModifiedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss

    def prediction_step(self, model: torch.nn.Module, inputs, prediction_loss_only: bool, ignore_keys = None):
        with torch.no_grad():
            res = model(
                input_ids=inputs["input_ids"].to(model.device),
                labels=inputs["labels"].to(model.device),
            ).loss
        return (res, None, None)

    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME

        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [tokenizer.pad_token_id] * (seq_len - 1) + ids[(seq_len - 1) :] + [tokenizer.pad_token_id] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback

In [ ]:
# Took about 10 compute unit

writer = SummaryWriter()
trainer = ModifiedTrainer(
    model=model,
    args=training_args,         # Trainer args
    train_dataset=dataset["train"],   # Training set
    eval_dataset=dataset["test"],    # Testing set
    data_collator=data_collator,     # Data Collator
    callbacks=[TensorBoardCallback(writer)]
)
trainer.train()
writer.close()

# save model
model.save_pretrained(training_args.output_dir)

You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
500,10.189500,6.481558


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


### 4.3 Model Saving and Download:
After training, save and download your model. You can also check the model's size.



In [ ]:
#!zip -r /content/saved_model.zip /content/{training_args.output_dir}
# download to local
#from google.colab import files
#files.download('/content/saved_model.zip')

  adding: content/./finetuned_model/ (stored 0%)
  adding: content/./finetuned_model/README.md (deflated 66%)
  adding: content/./finetuned_model/runs/ (stored 0%)
  adding: content/./finetuned_model/runs/Feb09_08-05-26_6e61fe3df392/ (stored 0%)
  adding: content/./finetuned_model/runs/Feb09_08-05-26_6e61fe3df392/events.out.tfevents.1707466661.6e61fe3df392.412.1 (deflated 58%)
  adding: content/./finetuned_model/adapter_model.safetensors (deflated 7%)
  adding: content/./finetuned_model/adapter_config.json (deflated 49%)
  adding: content/./finetuned_model/checkpoint-500/ (stored 0%)
  adding: content/./finetuned_model/checkpoint-500/optimizer.pt (deflated 8%)
  adding: content/./finetuned_model/checkpoint-500/trainer_state.json (deflated 52%)
  adding: content/./finetuned_model/checkpoint-500/rng_state.pth (deflated 25%)
  adding: content/./finetuned_model/checkpoint-500/training_args.bin (deflated 51%)
  adding: content/./finetuned_model/checkpoint-500/adapter_model.bin (deflated 48%

In [ ]:
# save to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save the finetuned model to google drive
!cp -r "/content/finetuned_model" "/content/drive/MyDrive"


In [ ]:
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, _, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / 1024 / 1024  # Size in MB

model_size = get_folder_size(training_args.output_dir)
print(f"Model size: {model_size} MB")


Model size: 2028.096851348877 MB


Now your model is trained and saved! You can download it and use it for generating financial insights or any other relevant tasks in the finance domain. The usage of TensorBoard allows you to deeply understand and visualize the training dynamics and performance of your model in real-time.

Happy FinGPT Training! 🚀

## Part 5: Inference and Benchmarks using FinGPT
Now that your model is trained, let’s understand how to use it to infer and run benchmarks.
* Took about 10 compute units



### 5.1 Load the model

In [ ]:
#clone the FinNLP repository
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

import sys
sys.path.append('/content/FinNLP/')

Cloning into 'FinNLP'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 1399 (delta 239), reused 431 (delta 224), pack-reused 941
Receiving objects: 100% (1399/1399), 4.97 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (612/612), done.


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

from peft import PeftModel
import torch

# Load benchmark datasets from FinNLP
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

In [ ]:
!pip install --upgrade peft

In [ ]:
# load model from google drive
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the path you want to check
path_to_check = "/content/drive/My Drive/finetuned_model"

# Check if the specified path exists
if os.path.exists(path_to_check):
    print("Path exists.")
else:
    print("Path does not exist.")


Path exists.


In [ ]:
## load the Baichuan base model
base_model = "baichuan-inc/Baichuan2-7B-Base"
peft_model = '/content/drive/My Drive/finetuned_model'

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True,padding_side='left')
model = AutoModelForCausalLM.from_pretrained(base_model,
                        quantization_config=q_config,
                        trust_remote_code=True,
                        device_map="auto")

model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 5.2 Run Benchmarks:

In [1]:
batch_size = 8

In [ ]:
# TFNS Test Set, len 2388
# Available: 84.85 compute units
res = test_tfns(model, tokenizer, batch_size = batch_size)
# Available: 83.75 compute units
# Took about 1 compute unite to inference



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 8. Total steps: 299


100%|██████████| 299/299 [01:00<00:00,  4.93it/s]


Acc: 0.6742043551088778. F1 macro: 0.3420289855072464. F1 micro: 0.6742043551088778. F1 weighted (BloombergGPT): 0.5579980036585314. 


In [ ]:
# FPB, len 1212
res = test_fpb(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 8. Total steps: 152


100%|██████████| 152/152 [01:00<00:00,  2.51it/s]


Acc: 0.6171617161716172. F1 macro: 0.3297609094589834. F1 micro: 0.6171617161716172. F1 weighted (BloombergGPT): 0.48406777169127635. 


In [ ]:
# FiQA, len 275
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 8. Total steps: 35


100%|██████████| 35/35 [00:07<00:00,  4.51it/s]

Acc: 0.10545454545454545. F1 macro: 0.10416666666666667. F1 micro: 0.10545454545454545. F1 weighted (BloombergGPT): 0.06446969696969697. 


In [ ]:
# NWGI, len 4047
res = test_nwgi(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/16184 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4047 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 8. Total steps: 506


100%|██████████| 506/506 [21:17<00:00,  2.52s/it]

Acc: 0.4252532740301458. F1 macro: 0.25262410905873534. F1 micro: 0.4252532740301458. F1 weighted (BloombergGPT): 0.2761419767998073. 
